In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import pathlib
import os



In [2]:
df = pd.read_csv('HousingPrices.csv')
X = df.drop(columns=['SalePrice'])
ydf = df[['SalePrice']]
df.head()



,YearBuilt,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,YrSold,SalePrice
0,2003,854,1710,2,1,3,8,2008,208500
1,1976,0,1262,2,0,3,6,2007,181500
2,2001,866,1786,2,1,3,6,2008,223500
3,1915,756,1717,1,0,3,7,2006,140000
4,2000,1053,2198,2,1,4,9,2008,250000


In [3]:

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, ydf, test_size = 0.2)

In [4]:
xval, x_test, yval, y_test = train_test_split(xtest, ytest, test_size = 0.5)


     

In [5]:

    
    model = keras.Sequential()
    model.add(keras.layers.Dense(units= 8,activation= 'relu', input_shape = (8,)))
    model.add(keras.layers.Dense(units= 8, activation= 'relu'))
    model.add(keras.layers.Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_squared_error'])
    model.summary()
   

Model: &quot;sequential&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [6]:
 model.fit(xtrain,ytrain, epochs=30,validation_data=(xval,yval))
    

Epoch 1/30
37/37 [==============================] - 0s 4ms/step - loss: 38458482688.0000 - mean_squared_error: 38458482688.0000 - val_loss: 40367620096.0000 - val_mean_squared_error: 40367620096.0000
Epoch 2/30
37/37 [==============================] - 0s 998us/step - loss: 38314094592.0000 - mean_squared_error: 38314094592.0000 - val_loss: 40187011072.0000 - val_mean_squared_error: 40187011072.0000
Epoch 3/30
37/37 [==============================] - 0s 1ms/step - loss: 38092304384.0000 - mean_squared_error: 38092304384.0000 - val_loss: 39919304704.0000 - val_mean_squared_error: 39919304704.0000
Epoch 4/30
37/37 [==============================] - 0s 1ms/step - loss: 37777444864.0000 - mean_squared_error: 37777444864.0000 - val_loss: 39558766592.0000 - val_mean_squared_error: 39558766592.0000
Epoch 5/30
37/37 [==============================] - 0s 964us/step - loss: 37359276032.0000 - mean_squared_error: 37359276032.0000 - val_loss: 39074156544.0000 - val_mean_squared_error: 39074156544.0

&lt;tensorflow.python.keras.callbacks.History at 0x173ebca4188&gt;

In [7]:
test_data = np.array([2003,	854,	1710,	2,	1,	3,	8,	2008])
print(model.predict(test_data.reshape(1,8)))

[[205640.81]]


In [8]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"


In [9]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\NAMANB~1\AppData\Local\Temp\tmp43mqwpb1\assets


In [10]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

2224

In [12]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [1 8]
Input Type: &lt;class &#39;numpy.float32&#39;&gt;
Output Shape: [1 1]
Output Type: &lt;class &#39;numpy.float32&#39;&gt;
